# Desafio AWS Glue Job

Analise atentamente os comandos a seguir e execute-os em sequência.

#### Mini tutorial Jupyter Notebook
Caso você não tenha familiaridade com Jupyter notebooks, aqui está um mini tutorial para começar:

**Para executar células**: Um notebook é composto por células. Cada célula pode conter código Python ou texto formatado. Para executar o código em uma célula, clique na célula para selecioná-la, depois pressione `Shift+Enter`. O resultado do código será exibido abaixo da célula.

**Para adicionar novas células**: Você pode adicionar novas células clicando no botão "+" na barra de ferramentas na parte superior. A nova célula será adicionada abaixo da célula atualmente selecionada.

**Salvar o notebook**: Você pode salvar o notebook clicando no botão de **Save** na parte superior.

**Interromper o notebook**: Se quiser sair, não esqueça de interromper o notebook utilizando o botão **Stop notebook**. Isso garante que os recursos alocados para o notebook sejam liberados e você não seja cobrado por isso.

### 1. Iniciando uma sessão interativa.


In [ ]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
import boto3
import time
from awsglue.transforms import *
from pyspark.sql.functions import sum, col, desc
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame  

print("Criando o contexto do Glue")
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
print("Contexto do Glue criado com sucesso")

### 2. Definindo a variável de ambiente `s3_bucket`

In [ ]:
print("Definindo a variável s3_bucket que vamos utilizar ao longo do código")
s3_bucket = ""
s3_client = boto3.client('s3')
response = s3_client.list_buckets()

for bucket in response['Buckets']:
    if bucket['Name'].startswith('lab-data-eng-'):
        s3_bucket = bucket['Name']

print("O bucket que vamos utilizar é: " + s3_bucket)

### 3. Criando um DynamicFrame a partir de uma tabela do Glue Data Catalog


In [ ]:
dyfPagamentos = glueContext.create_dynamic_frame.from_catalog(database='bolsafamilia', table_name='raw_pagamentos')

In [ ]:
dyfPagamentos.printSchema()

### 4. Apresentando algumas linhas

In [ ]:
dyfPagamentos.toDF().show(5)

### 5. Escrevendo os dados no S3 e criando a tabela `pagamentos_parquet` no Glue Data Catalog

In [ ]:
# Primeiro vamos limpar a pasta destino
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix='stage/bolsafamilia/pagamentos/')

for object in response['Contents']:
    out = s3_client.delete_object(Bucket=s3_bucket, Key=object['Key'])

In [ ]:
s3output = glueContext.getSink(
  path="s3://" + s3_bucket + "/stage/bolsafamilia/pagamentos/",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=["mes_competencia", "uf"],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="bolsafamilia", catalogTableName="pagamentos_parquet"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(dyfPagamentos)

### 6. Verificando os arquivos Parquet gerados

In [ ]:
session = boto3.Session()
s3 = session.resource('s3')
lab_eng_bucket = s3.Bucket(s3_bucket)

print("Arquivos de pagamentos em stage")
for clientes_object in lab_eng_bucket.objects.filter(Prefix="stage/bolsafamilia/pagamentos/"):
    print(clientes_object.key)

### 7. Verificando a criação da tabela `pagamentos_parquet`

> ##### Atenção!
> Normalmente esse tipo de verificação visual não fica dentro do job. Neste laboratório só incluí por comodidade.

In [ ]:
!aws glue get-tables --database-name 'bolsafamilia'  --query "TableList[].Name"

### 8. Relatório consolidado por Competência + UF

##### Criando os dynamicframes

In [ ]:
dyfPagamentos = glueContext.create_dynamic_frame.from_catalog(database='bolsafamilia', table_name='pagamentos_parquet')

In [ ]:
dyfPagamentos.printSchema()

In [ ]:
dyfPagamentos.toDF().show(5)

##### Tratamento do dataframe `dyfPagamentos`

Eliminando atributos desnecessários nesta operação

In [ ]:
dyfPagamentos = dyfPagamentos.drop_fields(["mes_referencia","codigo_municipio_siafi","municipio","cpf","nis","favorecido"])

In [ ]:
dyfPagamentos.toDF().show(5)

##### Calculando o valor consolidado de pagamentos por competência e estado

In [ ]:
# Converte para dataframe
dfPagamentos = dyfPagamentos.toDF()

In [ ]:
dfConsolidado = dfPagamentos.groupBy("mes_competencia", "uf") \
        .agg(sum("valor").alias("valor_total")) \
        .sort(desc("valor_total")) 

##### Salvando o resultado em uma tabela `tb_consolidado_pagamentos_competencia_uf` para reuso

In [ ]:
# Primeiro vamos limpar a pasta destino
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix='analytics/bolsafamilia/relatorios/consolidado_pagamentos_competencia_uf/')

for object in response['Contents']:
    out = s3_client.delete_object(Bucket=s3_bucket, Key=object['Key'])

In [ ]:
dyfConsolidado = DynamicFrame.fromDF(dfConsolidado, glueContext, "dyfConsolidado")

s3output = glueContext.getSink(
  path="s3://" + s3_bucket + "/analytics/bolsafamilia/relatorios/consolidado_pagamentos_competencia_uf/",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="bolsafamilia", catalogTableName="tb_consolidado_pagamentos_competencia_uf"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(dyfConsolidado)

# Parabéns!
Você concluiu a execução de um job Glue que pode ser empacotado e automatizado para execução periódica.<br>

> Não esqueça de clicar no botão **Stop notebook** para evitar cobranças extras.